In [1]:
import requests
import os
import pandas as pd
import json
import csv

### YELP-Foursquare_EDA

In [2]:
api_key = os.environ["location"]
client_id = os.environ["client_id"]
client_secret = os.environ["client_secret"]
api_version = '20230802'

In [3]:
location = "Vancouver, Canada"
url = "https://api.foursquare.com/v3/places/search?near=" + location

headers = {"Accept": "application/json"}

headers['Authorization'] = api_key

In [4]:
result = requests.get(url, headers=headers)
print(result)

<Response [200]>


In [5]:
data = result.json()
json_data = json.dumps(data, indent=4)
# print(json_data)

In [7]:
df_city_bike = pd.read_csv('city_bikes.csv')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [8]:
city_name = "Vancouver, Canada"

# '49.2827,-123.1207'
def foursquare_bike_stations(city_name, latitude, longitude, api_key):
    url = f"https://api.foursquare.com/v3/places/search"

    params = {
        'll': f"{latitude},{longitude}",  # Latitude and longitude of the bike station
        'radius': 500,
        'categories': "13033",   # foursquare category for bubble tea shops
        'limit': 5
    }

    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        bike_stations = data.get('results', [])

        return bike_stations
    else:
        print(f"Error: Unable to fetch Foursquare data. Status code: {response.status_code}")
        return []

# Function to collect the details of the bubble tea shops for each bike station
def bubble_tea_details(station_name, bubble_tea_shops):
    details_list = []
    for idx, shop in enumerate(bubble_tea_shops, start=1):
        name = shop.get('name', 'N/A')
        rating = shop.get('rating', 'N/A')
        location = shop.get('location', {})
        shop_latitude = location.get('lat', 'N/A')
        shop_longitude = location.get('lng', 'N/A')

        details_list.append({
            'Station Name': station_name,
            'Bubble Tea Shop Name': name,
            'Rating': rating,
            'Latitude': shop_latitude,
            'Longitude': shop_longitude
        })

    return details_list

In [9]:
# Create an empty list to store the details of bubble tea shops for all bike stations
all_bubble_tea_details = []

# Loop through each bike station and get bubble tea shop details
for index, row in df_city_bike.iterrows():
    station_name = row['Station Name']
    latitude = row['Latitude']
    longitude = row['Longitude']

    bubble_tea_shops = foursquare_bike_stations(city_name, latitude, longitude, api_key)

    # Collect bubble tea shop details for the current bike station
    station_bubble_tea_details = bubble_tea_details(station_name, bubble_tea_shops)

    # Add the details to the list
    all_bubble_tea_details.extend(station_bubble_tea_details)

# Create a DataFrame from the collected bubble tea shop details
df_bubble_tea = pd.DataFrame(all_bubble_tea_details)

# Print DataFrame
df_bubble_tea

,Station Name,Bubble Tea Shop Name,Rating,Latitude,Longitude
0,10th & Cambie,OneZo Tapioca,N/A,N/A,N/A
1,Yaletown-Roundhouse Station,ShareTea Yaletown,N/A,N/A,N/A
2,Yaletown-Roundhouse Station,Chabann Kurotaki,N/A,N/A,N/A
3,Yaletown-Roundhouse Station,Cha Yuan,N/A,N/A,N/A
4,Dunsmuir & Beatty,Gong Cha,N/A,N/A,N/A
...,...,...,...,...,...
341,Cordova & Granville,Coco Fresh Tea & Juice,N/A,N/A,N/A
342,Cordova & Granville,Teaever Bubble Tea,N/A,N/A,N/A
343,Cordova & Granville,Gong Cha West Pender,N/A,N/A,N/A
344,Cordova & Granville,Chatime,N/A,N/A,N/A


In [10]:
# Export Foursquare DF to csv to use in later code
df_foursquare = 'foursquare.csv'
df_bubble_tea.to_csv(df_foursquare, index=False)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [11]:
import requests
import os

yelp_api_key = os.environ["yelp_api_key"]

url = "https://api.yelp.com/v3/categories"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {yelp_api_key}"
}

response = requests.get(url, headers=headers)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [13]:
city_name = "Vancouver, Canada"

def yelp_bike_stations(city_name, latitude, longitude, yelp_api_key):
    url = f"https://api.yelp.com/v3/businesses/search"

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': 500,              # 500 meters radius for Yelp API request - reduced to return less results, run faster cod
        'categories': "bubbletea",
        'limit': 5
    }

    headers = {"Authorization": f"Bearer {yelp_api_key}"}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        bike_stations = data.get('businesses', [])

        return bike_stations
    else:
        print(f"Error: Unable to fetch Yelp data. Status code: {response.status_code}")
        return []

# collect details of Yelp businesses for each bike station
def collect_yelp_details(station_name, bike_stations):
    details_list = []
    for idx, station in enumerate(bike_stations, start=1):
        station_latitude = station.get('coordinates', {}).get('latitude', 'N/A')
        station_longitude = station.get('coordinates', {}).get('longitude', 'N/A')

        # Fetch Yelp business details for the current bike station
        yelp_businesses = yelp_bike_stations(city_name, station_latitude, station_longitude, yelp_api_key)

        if yelp_businesses:
            for idx, business in enumerate(yelp_businesses, start=1):
                name = business.get('name', 'N/A')
                rating = business.get('rating', 'N/A')
                business_latitude = business.get('coordinates', {}).get('latitude', 'N/A')
                business_longitude = business.get('coordinates', {}).get('longitude', 'N/A')

                details_list.append({
                    'Station Name': station_name,
                    'Station Latitude': station_latitude,
                    'Station Longitude': station_longitude,
                    'Bubble Tea Shop Name': name,                    # Yelp Business Name
                    'Rating': rating,
                    'Business Latitude': business_latitude,
                    'Business Longitude': business_longitude
                })

    return details_list

In [14]:
# Create an empty list to store the details of Yelp businesses for all bike stations
all_yelp_details = []

# Loop through each bike station and get Yelp business details
for index, row in df_city_bike.iterrows():
    station_name = row['Station Name']
    latitude = row['Latitude']
    longitude = row['Longitude']

    bike_stations = yelp_bike_stations(city_name, latitude, longitude, yelp_api_key)

    # Collect Yelp business details for the current bike station
    station_yelp_details = collect_yelp_details(station_name, bike_stations)

    # Add the details to the list
    all_yelp_details.extend(station_yelp_details)

# Create a DataFrame from the collected Yelp business details
df_yelp_businesses = pd.DataFrame(all_yelp_details)

# Print DF
df_yelp_businesses

Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp data. Status code: 429
Error: Unable to fetch Yelp dat

,Station Name,Station Latitude,Station Longitude,Bubble Tea Shop Name,Rating,Business Latitude,Business Longitude
0,10th & Cambie,49.259935,-123.114855,Sharetea,4.0,49.259935,-123.114855
1,10th & Cambie,49.259935,-123.114855,Bubble Waffle Cafe,4.0,49.260860,-123.116290
2,10th & Cambie,49.259935,-123.114855,Hot Cha Cafe,4.5,49.255697,-123.114906
3,10th & Cambie,49.259935,-123.114855,OneZo Tapioca,4.5,49.263037,-123.118080
4,10th & Cambie,49.259935,-123.114855,Ben Gong’s Tea,3.5,49.255610,-123.114909
...,...,...,...,...,...,...,...
1312,Richards & Davie,49.276097,-123.119294,Teaspoons & Company,4.5,49.279506,-123.124349
1313,Richards & Davie,49.276097,-123.119294,Cha Yuan,4.5,49.274020,-123.122430
1314,Richards & Davie,49.276097,-123.119294,Jenjudan Yaletown,4.0,49.278330,-123.116210
1315,Richards & Davie,49.276097,-123.119294,Gong Cha,3.0,49.279203,-123.116752


In [15]:
# Export Yelp DF to use in later code
yelp_businesses = 'yelp_businesses.csv'

df_yelp_businesses.to_csv(yelp_businesses, index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

##### The YELP API was incredibly more complete. Setting up the code to be similar in both Foursquare and YELP, I found that the YELP one listed all the names, rating, both latitude and longitude, whereas the Foursquare only listed the bubble tea stores by name and returned N/A for rating and both latitude and longitude. 

Get the top 10 restaurants according to their rating (bubbletea shops)

In [16]:
# Use previously exported DF so that our code isn't as taxed

df_yelp_businesses = pd.read_csv('yelp_businesses.csv')

In [18]:
# Sort the DataFrame by 'Yelp Rating' in descending order
sorted_df = df_yelp_businesses.sort_values(by='Rating', ascending=False)

# Get the top 10 bubbletea shops according to their rating
top_10_bbt_shops = sorted_df.nlargest(10, 'Rating')

# Print the top 10 restaurants
top_10_bbt_shops

,Station Name,Station Latitude,Station Longitude,Bubble Tea Shop Name,Rating,Business Latitude,Business Longitude
635,Nelson & Mainland,49.278330,-123.116210,Gong cha,5.0,49.279050,-123.116950
452,Bute & Comox,49.280832,-123.124743,Tea Drop Café,5.0,49.279501,-123.124057
152,Marinaside & Davie,49.274020,-123.122430,Sharetea Yaletown,5.0,49.276097,-123.119294
401,Pacific & Richards,49.274020,-123.122430,Sharetea Yaletown,5.0,49.276097,-123.119294
899,Richards & Helmcken,49.274020,-123.122430,Sharetea Yaletown,5.0,49.276097,-123.119294
895,Richards & Helmcken,49.279506,-123.124349,Tea Drop Café,5.0,49.279501,-123.124057
406,Pacific & Richards,49.274646,-123.125849,Sharetea Yaletown,5.0,49.276097,-123.119294
892,Beach & Seymour,49.274646,-123.125849,Sharetea Yaletown,5.0,49.276097,-123.119294
885,David Lam Park - West,49.274646,-123.125849,Sharetea Yaletown,5.0,49.276097,-123.119294
880,David Lam Park - West,49.274020,-123.122430,Sharetea Yaletown,5.0,49.276097,-123.119294
